**Note:** If running at DTU HPC, remember to set `n_jobs` to something else than `-1`!

In [ ]:
n_jobs = -1

In [1]:
import os
drive_dir = '/work3/s184399/msc'
data_dir = os.path.join(drive_dir, 'Detection-datasets')
cache_dir = os.path.join(drive_dir, "cache_dir", "huggingface")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
from sklearn.model_selection import GridSearchCV, cross_validate, StratifiedGroupKFold, GroupShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from scipy.stats import beta
from typing import Dict, List, Tuple
from time import time
import seaborn as sns
from tqdm.notebook import tqdm
import pandas as pd


linear_svm_params = {
  'SVC__C': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0],
  'SVC__kernel': ['linear'],
}
rbf_svm_params = {
  'SVC__C': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0],
  'SVC__kernel': ['rbf'],
}
debug_svm_params = {
    'HeadSelector__layer_ix': [8,9],
    'HeadSelector__head_ix': [8,9],
    'SVC__C': [0.1],
    'SVC__kernel': ['linear'],
}


class HeadSelector(BaseEstimator, TransformerMixin):
  def __init__(self, layer_ix=None, head_ix=None):
    self.layer_ix = layer_ix
    self.head_ix = head_ix

  def fit(self, X: np.array, y:None):
    return self

  def transform(self, features: np.ndarray) -> np.ndarray:
    # Features has shape: [num_layers, num_samples, num_heads, head_dim]
    out = features[:,self.layer_ix,self.head_ix,:]
    assert out.ndim == 2
    return out   # [num_samples, head_dim]


def make_joint_plot(activations, labels, layer_ix, head_ix, title):
  head_selector = HeadSelector(layer_ix, head_ix)
  X = head_selector.transform(activations)
  y = labels

  # Greedily find the two directions with highest predictability.
  first_direction = SVC(kernel='linear')
  second_direction = SVC(kernel='linear')
  first_direction.fit(X,y)
  w1 = first_direction.coef_.T
  proj_1 = X @ w1
  X_minus_proj_1 = X - proj_1/(w1.T.dot(w1))*w1.reshape(1,-1)
  assert np.isclose(X_minus_proj_1@w1,0.).all()    # Check that it is orthogonal.
  second_direction.fit(X_minus_proj_1,y)
  w2 = second_direction.coef_.T
  proj_2 = X @ w2

  d = pd.DataFrame({
      "SVM direction 1": proj_1.ravel(),
      "SVM direction 2": proj_2.ravel(),
      "label": y.ravel().astype(bool)
  })
  sns.set_theme()    # https://seaborn.pydata.org/tutorial/introduction.html and https://seaborn.pydata.org/generated/seaborn.kdeplot.html
  #sns.kdeplot(data=d, x="P1", y="P2", hue="label", fill=True)#, levels=5, thresh=.2)
  g = sns.jointplot(data=d, x="SVM direction 1", y="SVM direction 2", hue="label", marker='.')
  g.plot_joint(sns.kdeplot, zorder=0, levels=3)

  g.fig.suptitle(title)
  g.ax_joint.collections[0].set_alpha(0)
  g.fig.tight_layout()
  g.fig.subplots_adjust(top=0.95) # Reduce plot to make room


def CV_experiment(activations, labels, param_grid, title, ood_test_sets:List[Tuple[np.ndarray, np.ndarray]]=[], groups=None, n_inner_folds=5, ci_alpha = 0.05, n_jobs=-1):
  """
    Runs 2-layer CV on an SVM probe with a set kernel.
    The CV setup is like this:
    - For the outer loop, we use (repeated) hold-out - i.e. split in dev and test set.
    - For the inner loop, we use K-fold CV to pick the best scoring parameters. Note that the best classifier is then retrained by Scikit-Learn to predict on the test set.
    The CV is grouped when groups are given. It defaults to non-grouped CV.
    Using the test set, we finally compute the Jeffrey Intervals.

    Args:
      - activations: The activations in this format [num_samples, num_layers, num_heads, head_dim]
      - labels: The labels for the statements corresponding to the activations in shape [num_samples,]
      - param_grid: Parameter grid for the probes. Adheres to Scikit-Learns documentation for GridSearchCV. Be sure to include the layers and heads in this one!
          Example:
            param_grid = {
              'HeadSelector__layer_ix': list(range(0,activations.shape[1])),     # If not specified, it will automatically insert them.
              'HeadSelector__head_ix': list(range(0,activations.shape[2])),
              'SVC__C': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0],
              'SVC__kernel': ['rbf'],
            }

      - ood_test_sets: Out-of-distribution test sets which we will also evaluate on by using the probe trained on the dev_set (i.e. it is NOT retrained).
          Has format: [(X,y),(X,y),(X,y),...] where each (X,y) is a separate test set.
      - groups: For how to group the data in the CV to ensure no information leakage.
      - ci_alpha: The alpha value used in computing the 1-alpha (central) confidence intervals.
      - n_inner_folds: Number of inner CV folds
      - n_jobs: The number of parallel jobs to run. Defaults to -1 meaning ALL available workers (set it to less if running at HPC!!!)
  """
  n_outer_holdout_repeats = 1  # Should not be changed without modifying the CI computation.

  if not 'HeadSelector__layer_ix' in param_grid.keys():
    param_grid['HeadSelector__layer_ix'] = list(range(0,activations.shape[1]))
  if not 'HeadSelector__head_ix' in param_grid.keys():
    param_grid['HeadSelector__head_ix'] = list(range(0,activations.shape[2]))

  if groups is None:
    groups = np.arange(len(labels))   # I.e. each instance becomes a group = no grouping

  inner_cv = StratifiedGroupKFold(n_splits=n_inner_folds)
  outer_cv = StratifiedGroupKFold(n_splits=2)    # Implement holdout by breaking after this

  probe_pipeline = Pipeline([
      ('HeadSelector', HeadSelector()),   # Head selector's layer_ix, and head_ix are set in CV.
      ('StandardScaler', StandardScaler()),
      ('SVC', SVC())
  ])
  cv = GridSearchCV(
    probe_pipeline, param_grid, cv=inner_cv, refit=True, n_jobs=n_jobs, verbose=4,
  )

  # Cross validate. Outer loop: Holdout, Inner loop: k-fold
  start_time = time()
  correct_guesses = 0
  for dev_idx, test_idx in tqdm(outer_cv.split(activations, labels, groups=groups), desc="Outer CV", total=n_outer_holdout_repeats, leave=True):
    # Assert that the dev set and test set partition the entire dataset correctly based on groups
    assert len(set(groups[dev_idx]).intersection(set(groups[test_idx]))) == 0  # Disjoint
    assert (len(dev_idx) + len(test_idx)) == len(labels)  # Covers entire dataset

    # Inner CV: K-fold
    cv.fit(activations[dev_idx,:,:,:], labels[dev_idx], groups=groups[dev_idx])
    ## FUTURE NOTE - These exist: cv.cv_results_['params'], cv.cv_results_['split0_test_score'], cv.cv_results_['mean_test_score']

    layer_ix, head_ix = cv.best_params_['HeadSelector__layer_ix'], cv.best_params_['HeadSelector__head_ix']

    # Predict on the test set
    pred = cv.predict(activations[test_idx,:,:,:])   # Note: the best model is refitted by Scikit-Learn.
    correct_guesses += np.sum(pred == labels[test_idx])

    break   # Implement holdout by breaking

  make_joint_plot(activations, labels, layer_ix, head_ix, title)

  # Compute Jeffrey CI's:
  N = len(test_idx)
  m = correct_guesses
  a = m+0.5
  b = N-m+0.5
  acc_est = m/N
  CI = np.array([beta.ppf(q=q, a=a, b=b) for q in [ci_alpha/2., 1-ci_alpha/2.]])
  return_dict = {
      "test_acc": acc_est,
      "test_acc_CI": CI,
      "test_acc_support": N,
      "test_n_correct_predictions": m
  }
  end_time = time()
  return_dict['CV_time'] = end_time-start_time   # In seconds

  # Evaluate the trained best probe on the alternative test sets:
  if len(ood_test_sets) > 0:
    for i, (X, y) in enumerate(ood_test_sets):
      pred = cv.predict(X)
      m = np.sum(pred == y)
      N = len(y)
      a = m+0.5
      b = N-m+0.5
      acc_est = m/N
      CI = np.array([beta.ppf(q=q, a=a, b=b) for q in [ci_alpha/2., 1-ci_alpha/2.]])
      return_dict[f'ood_test_{i}_acc'] = acc_est
      return_dict[f'ood_test_{i}_acc_CI'] = CI
      return_dict[f'ood_test_{i}_acc_support'] = N
      return_dict[f'ood_test_{i}_n_correct_predictions'] = m

  return return_dict

In [3]:
import os
datasets = list(ds_name.replace('_prepared.csv', '') for ds_name in filter(lambda x: '.csv' in x, os.listdir(data_dir)))
models = ["facebook/opt-125m", "facebook/opt-350m", "facebook/opt-1.3b", "facebook/opt-2.7b", "meta-llama/Llama-2-7b-chat-hf", "meta-llama/Llama-2-13b-chat-hf", "meta-llama/Llama-2-70b-chat-hf", "microsoft/Phi-3-mini-4k-instruct", "meta-llama/Meta-Llama-3-8B", "meta-llama/Meta-Llama-3-70B-Instruct"]

truthful_qa_ = "TruthfulQA"
datasets_ = list(filter(lambda x: 'truthfulqa' not in x.lower(), datasets))
datasets_

['Politicians', 'neg_cities', 'common_claim_true_false', 'cities']

## Train on TruthfulQA

### Linear SVM

In [ ]:
import pandas as pd
from IPython.display import display, clear_output

truthful_qa_df = pd.read_csv(os.path.join(data_dir, f"{truthful_qa_}_prepared.csv"))
truthful_qa_labels = truthful_qa_df['isTrue']
truthful_qa_groups = truthful_qa_df['Group']

performance_df = pd.DataFrame(columns=['TruthfulQA'] + datasets_)

for i, model in tqdm(enumerate(models), total=len(models), desc='Models', leave=False):
  model_name_str = model.split('/')[-1]
  truthful_qa_activations = np.load(os.path.join(data_dir, f"Activations_{truthful_qa_}_{model_name_str}.npz"))['arr_0']
  ood_test_sets = [
      (
          np.load(os.path.join(data_dir, f"Activations_{ds_name}_{model_name_str}.npz"))['arr_0'],
          pd.read_csv(os.path.join(data_dir, f"{ds_name}_prepared.csv"))['isTrue']
      )  # (X,y)
      for ds_name in datasets_
  ]
  results = CV_experiment(
      activations=truthful_qa_activations,
      labels=truthful_qa_labels,
      groups=truthful_qa_groups,
      param_grid=linear_svm_params,
      ood_test_sets=ood_test_sets,
      title=f"{model_name_str} on TruthfulQA",
      n_jobs=n_jobs,
  )
  performance_df.loc[i] = [results['test_acc_CI']] + [results[f'ood_test_{i}_acc_CI'] for i in range(len(datasets_))]
  clear_output(wait=True)
  display(performance_df)
  print(f"Time for {model_name_str}: {results['CV_time']:.2f}s")

performance_df

Models:   0%|          | 0/6 [00:00<?, ?it/s]

Outer CV:   0%|          | 0/1 [00:00<?, ?it/s]

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


### RBF SVM

In [ ]:
import pandas as pd
from IPython.display import display, clear_output

truthful_qa_df = pd.read_csv(os.path.join(data_dir, f"{truthful_qa_}_prepared.csv"))
truthful_qa_labels = truthful_qa_df['isTrue']
truthful_qa_groups = truthful_qa_df['Group']

performance_df = pd.DataFrame(columns=['TruthfulQA'] + datasets_)

for i, model in tqdm(enumerate(models), total=len(models), desc='Models', leave=False):
  model_name_str = model.split('/')[-1]
  truthful_qa_activations = np.load(os.path.join(data_dir, f"Activations_{truthful_qa_}_{model_name_str}.npz"))['arr_0']
  ood_test_sets = [
      (
          np.load(os.path.join(data_dir, f"Activations_{ds_name}_{model_name_str}.npz"))['arr_0'],
          pd.read_csv(os.path.join(data_dir, f"{ds_name}_prepared.csv"))['isTrue']
      )  # (X,y)
      for ds_name in datasets_
  ]
  results = CV_experiment(
      activations=truthful_qa_activations,
      labels=truthful_qa_labels,
      groups=truthful_qa_groups,
      param_grid=rbf_svm_params,
      ood_test_sets=ood_test_sets,
      title=f"{model_name_str} on TruthfulQA",
      n_jobs=n_jobs,
  )
  performance_df.loc[i] = [results['test_acc_CI']] + [results[f'ood_test_{i}_acc_CI'] for i in range(len(datasets_))]
  clear_output(wait=True)
  display(performance_df)
  print(f"Time for {model_name_str}: {results['CV_time']:.2f}s")

performance_df

## Train on respective datasets

### Linear SVM

In [ ]:
datasets_ = ['TruthfulQA'] + datasets_

In [ ]:
import pandas as pd
from IPython.display import display, clear_output

performance_df = pd.DataFrame(columns=datasets_)

for i, model in tqdm(enumerate(models), total=len(models), desc='Models', leave=False):
  row = []
  for ds_name in datasets_:
    model_name_str = model.split('/')[-1]
    truthful_qa_activations = np.load(os.path.join(data_dir, f"Activations_{truthful_qa_}_{model_name_str}.npz"))['arr_0']
    d = [
        (
            np.load(os.path.join(data_dir, f"Activations_{ds_name}_{model_name_str}.npz"))['arr_0'],
            pd.read_csv(os.path.join(data_dir, f"{ds_name}_prepared.csv"))['isTrue']
        )  # (X,y)
        for ds_name in datasets_
    ]
    activations = np.load(os.path.join(data_dir, f"Activations_{ds_name}_{model_name_str}.npz"))['arr_0']
    df = pd.read_csv(os.path.join(data_dir, f"{ds_name}_prepared.csv"))
    labels = df['isTrue']
    groups = None
    if 'Group' in df.columns:
      groups = df['Group']
    results = CV_experiment(
        activations=activations,
        labels=labels,
        groups=groups,
        param_grid=linear_svm_params,
        ood_test_sets=None,
        title=f"{model_name_str} on {ds_name}",
        n_jobs=n_jobs,
    )
    row.append(results['test_acc_CI'])


  performance_df.loc[i] = row
  clear_output(wait=True)
  display(performance_df)
  print(f"Time for {model_name_str}: {results['CV_time']:.2f}s")

performance_df

### RBF SVM

In [ ]:
import pandas as pd
from IPython.display import display, clear_output

performance_df = pd.DataFrame(columns=datasets_)

for i, model in tqdm(enumerate(models), total=len(models), desc='Models', leave=False):
  row = []
  for ds_name in datasets_:
    model_name_str = model.split('/')[-1]
    truthful_qa_activations = np.load(os.path.join(data_dir, f"Activations_{truthful_qa_}_{model_name_str}.npz"))['arr_0']
    d = [
        (
            np.load(os.path.join(data_dir, f"Activations_{ds_name}_{model_name_str}.npz"))['arr_0'],
            pd.read_csv(os.path.join(data_dir, f"{ds_name}_prepared.csv"))['isTrue']
        )  # (X,y)
        for ds_name in datasets_
    ]
    activations = np.load(os.path.join(data_dir, f"Activations_{ds_name}_{model_name_str}.npz"))['arr_0']
    df = pd.read_csv(os.path.join(data_dir, f"{ds_name}_prepared.csv"))
    labels = df['isTrue']
    groups = None
    if 'Group' in df.columns:
      groups = df['Group']
    results = CV_experiment(
        activations=activations,
        labels=labels,
        groups=groups,
        param_grid=rbf_svm_params,
        ood_test_sets=None,
        title=f"{model_name_str} on {ds_name}",
        n_jobs=n_jobs,
    )
    row.append(results['test_acc_CI'])


  performance_df.loc[i] = row
  clear_output(wait=True)
  display(performance_df)
  print(f"Time for {model_name_str}: {results['CV_time']:.2f}s")

performance_df